<a href="https://colab.research.google.com/github/CRAUGUTH/aiProject/blob/main/Music_Organization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training NN**


## Installing and Importing Libraries

In [ ]:
# Task 1: Install Libraries
!pip install --upgrade spotipy torch pandas scikit-learn transformers librosa gdown

# Task 2: Import Libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import joblib
import gdown

## Defining the Model

As described within my progress report 2, I came across many issues when it came to using/implimented pre-trained models on HuggingFace. Because of these probelms I desided to train my own model to classify music based on emotion. My first model was linear, resulting in only a 50% success rate, thus I desided to add multiple layers to the model.

In [21]:
# Task 3: Define Your Model with a modified architecture and dropout
class SongMoodClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SongMoodClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.network(x)

## Load and Preprocess Data

Since I had to create my own model to classify music, that means that I also had to find my own data to train the model. This resulting in me searching the web for a dataset which contained the spotify song_id, mood, and the spotify song features. While searching for this I came across a dataset on Kaggle which conatined all this information

Kaggle Link: https://www.kaggle.com/code/muhammadghazimuharam/music-mood-classification

I then downloaded this data to Google Sheets where I then cleaned it up a bit and then uploaded it to my Google Colab Notebook

In [22]:
# Task 4: Load and Preprocess Data
df = pd.read_csv('/content/music_moods.csv')
features = df.drop(['id', 'mood'], axis=1).values
labels = df['mood'].values

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_val, y_train, y_val = train_test_split(features_scaled, labels_encoded, test_size=0.2, random_state=42)

## Train and Validate the Model

Here I am  training the model based on the data from the csv file then I am also validating it to make sure the model is atleast 80% accurate. Some modification that I made was that the training will now stop if the accuracy stops improving, then it'll also show you the final accuracy of the model once the training is complete.

In [30]:
# Task 5: Train and Validate the Model
input_size = X_train.shape[1]
num_classes = len(np.unique(labels_encoded))

model = SongMoodClassifier(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # Adjusted learning rate

# Adding a learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

batch_size = 65  # Adjust if needed

# Function to create batches
def create_batches(X, y, batch_size):
    for i in range(0, len(X), batch_size):
        yield X[i:i+batch_size], y[i:i+batch_size]

# Early stopping criteria
early_stopping_patience = 10
no_improvement_epochs = 0
best_val_accuracy = 0

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in create_batches(X_train, y_train, batch_size):
        inputs = torch.FloatTensor(X_batch)
        targets = torch.LongTensor(y_batch)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation loop with early stopping
    model.eval()
    val_losses = []
    val_accuracies = []
    for X_batch, y_batch in create_batches(X_val, y_val, batch_size):
        inputs = torch.FloatTensor(X_batch)
        targets = torch.LongTensor(y_batch)
        with torch.no_grad():
            outputs = model(inputs)
            val_loss = criterion(outputs, targets)
            val_losses.append(val_loss.item())

            _, predicted = torch.max(outputs, 1)
            val_accuracy = (predicted == targets).sum().item() / targets.size(0)
            val_accuracies.append(val_accuracy)

    avg_val_loss = np.mean(val_losses)
    avg_val_accuracy = np.mean(val_accuracies)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}, Validation Accuracy: {avg_val_accuracy}')

    # Check if current validation accuracy is better and reset no_improvement_epochs
    if avg_val_accuracy > best_val_accuracy:
        best_val_accuracy = avg_val_accuracy
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= early_stopping_patience:
        print("Stopping early due to no improvement")
        break

    # Update the learning rate
    scheduler.step()

# After the training loop, print the final accuracy
print(f'Final Validation Accuracy: {best_val_accuracy:.4f}')

Epoch 1/100, Validation Loss: 1.0300576289494832, Validation Accuracy: 0.5826923076923077
Epoch 2/100, Validation Loss: 0.7119933366775513, Validation Accuracy: 0.6705128205128205
Epoch 3/100, Validation Loss: 0.551711748043696, Validation Accuracy: 0.7480769230769231
Epoch 4/100, Validation Loss: 0.4758959114551544, Validation Accuracy: 0.7737179487179487
Epoch 5/100, Validation Loss: 0.4409123162428538, Validation Accuracy: 0.783974358974359
Epoch 6/100, Validation Loss: 0.42175376911958057, Validation Accuracy: 0.7891025641025641
Epoch 7/100, Validation Loss: 0.41086926062901813, Validation Accuracy: 0.7891025641025641
Epoch 8/100, Validation Loss: 0.4105342427889506, Validation Accuracy: 0.7942307692307692
Epoch 9/100, Validation Loss: 0.41039355595906574, Validation Accuracy: 0.7993589743589743
Epoch 10/100, Validation Loss: 0.40862559775511426, Validation Accuracy: 0.7942307692307692
Epoch 11/100, Validation Loss: 0.41942672928174335, Validation Accuracy: 0.8096153846153845
Epoch

Here I am just saving the model so that I can use it to classify songs from one of my spotify playlists

In [31]:
# Task 6: Save the Model and Preprocessors
torch.save(model.state_dict(), 'song_mood_classifier.pth')
joblib.dump(label_encoder, 'label_encoder.joblib')
joblib.dump(scaler, 'scaler.joblib')

['scaler.joblib']

# **Music Classification**

## Getting personal playlist for model to classify

Get new access token for Spotify account

In [39]:
client = spotipy.oauth2.SpotifyOAuth(
    client_id='cce8687d5317494cb81ff7731ccd9a2b',
    client_secret='9646e0745dde4bddb7c67b29431a61f0',
    redirect_uri='https://example.com/callback/',
    scope='playlist-modify-private'
)

sp = spotipy.Spotify(auth_manager=client)

# Get the authorization URL.
authorization_url = client.get_authorize_url()

# Redirect the user to the authorization URL.
print(authorization_url)

https://accounts.spotify.com/authorize?client_id=cce8687d5317494cb81ff7731ccd9a2b&response_type=code&redirect_uri=https%3A%2F%2Fexample.com%2Fcallback%2F&scope=playlist-modify-private


Authorize account

In [ ]:
code = "AQDvEwjq04nIsUnMNM04a1ikfNvTiNhKko1l9kOR-mkQhLtmDF6-e_PU52hY0X2Ny4DO3IHRqHqZuL6SnR-iuPnck8c8Rw528tf_ikxorDDD3TdjF1-Hxhmc0wGvkFeUr9zdSg8e5WFDzYizIyvp3HFnH0HNXx2kYS2xYhu3y0ZYPUB-wtL-m_C0aYNKjOUhl4DCDeGHNAQC6w"
client.get_access_token(code)

Getting playlist from my account I want to classify which is the USA Top Songs

In [42]:
# Get the playlist's ID
playlist_id = '37i9dQZEVXbLp5XoPON0wI'

# Get the playlist's tracks
tracks = sp.playlist_tracks(playlist_id)['items']

# Create a list of song IDs
song_ids = []
for track in tracks:
    song_ids.append(track['track']['id'])

## Classifying each song in playlist

Here I am creating a dictionary of the songs within that playlist. The dictionary contains the key, which is the song_id, and then the song name, artist, and mood.

In [43]:
results = {}
features_to_use = [
    'acousticness', 'danceability', 'energy', 'instrumentalness',
    'key', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'
]

for song_id in song_ids:
    # Retrieve song features from Spotify
    song_features = sp.audio_features(song_id)[0]

    # Retrieve song details from Spotify
    track_info = sp.track(song_id)
    song_name = track_info['name']
    artist_name = track_info['artists'][0]['name']  # Assume there is at least one artist

    # Prepare the input tensor for the model
    inputs = [song_features[feature] for feature in features_to_use]
    inputs_tensor = torch.tensor([inputs], dtype=torch.float32)

    # Classify the mood of the song using the model
    with torch.no_grad():
        outputs = model(inputs_tensor)
        _, predicted_class = torch.max(outputs, 1)
        predicted_label = label_encoder.inverse_transform([predicted_class.item()])[0]

    # Store the results including song name, artist, and mood
    results[song_id] = {
        'song_name': song_name,
        'artist_name': artist_name,
        'mood': predicted_label
    }

# Print the classification results along with song name and artist
for song_id, info in results.items():
    print(f"{info['song_name']} by {info['artist_name']}, Mood: {info['mood']}")

Rockin' Around The Christmas Tree by Brenda Lee, Mood: Happy
Lovin On Me by Jack Harlow, Mood: Energetic
All I Want for Christmas Is You by Mariah Carey, Mood: Energetic
Jingle Bell Rock by Bobby Helms, Mood: Energetic
My Love Mine All Mine by Mitski, Mood: Happy
Last Christmas - Single Version by Wham!, Mood: Energetic
It's the Most Wonderful Time of the Year by Andy Williams, Mood: Energetic
Stick Season by Noah Kahan, Mood: Energetic
I Remember Everything (feat. Kacey Musgraves) by Zach Bryan, Mood: Energetic
greedy by Tate McRae, Mood: Energetic
Agora Hills by Doja Cat, Mood: Energetic
Let It Snow! Let It Snow! Let It Snow! by Dean Martin, Mood: Energetic
A Holly Jolly Christmas by Burl Ives, Mood: Energetic
Is It Over Now? (Taylor's Version) (From The Vault) by Taylor Swift, Mood: Energetic
Cruel Summer by Taylor Swift, Mood: Energetic
It's Beginning to Look a Lot like Christmas by Michael Bublé, Mood: Energetic
500lbs by Lil Tecca, Mood: Energetic
IDGAF (feat. Yeat) by Drake, Moo

# **Playlist Generation**

Here I have the functions that I use within the UI below

In [45]:
def display_songs_by_mood(results, mood):
    matching_songs = [(song_id, info) for song_id, info in results.items() if info['mood'].lower() == mood.lower()]

    if not matching_songs:
        print(f"No songs found with the mood: {mood}")
        return []
    else:
        print(f"Songs with the mood '{mood}':")
        for song_id, info in matching_songs:
            print(f"{info['song_name']}, by {info['artist_name']}")
        return [song_id for song_id, _ in matching_songs]

def create_playlist(sp, user_id, song_ids, playlist_name):
    # Create a new playlist
    playlist = sp.user_playlist_create(user_id, playlist_name, public=False, description=f'Playlist for {playlist_name} mood')
    playlist_id = playlist['id']

    # Add songs to the playlist
    sp.playlist_add_items(playlist_id, song_ids)
    print(f'Playlist "{playlist_name}" created with {len(song_ids)} songs.')

# Get the current user's ID
user_id = sp.current_user()['id']

Here is the UI of the program, you can either display the songs of a certain mood and/or create a playlist out of the songs classified under a certain mood.

In [ ]:
# Interactive menu
while True:
    print("\nInteractive Music Mood Search:")
    print("1. Search for Happy songs")
    print("2. Search for Calm songs")
    print("3. Search for Sad songs")
    print("4. Search for Energetic songs")
    print("5. Create playlist from mood")
    print("0. Exit")

    choice = input("Enter your choice (0-5): ")

    if choice == '0':
        print("Exiting the program")
        break
    elif choice in ['1', '2', '3', '4']:
        mood_dict = {'1': 'Happy', '2': 'Calm', '3': 'Sad', '4': 'Energetic'}
        song_ids = display_songs_by_mood(results, mood_dict[choice])
    elif choice == '5':
        mood = input("Enter the mood for which to create a playlist: ")
        song_ids = display_songs_by_mood(results, mood)
        if song_ids:
            playlist_name = f"{mood} Mood Songs"
            create_playlist(sp, user_id, song_ids, playlist_name)
    else:
        print("Invalid choice. Please enter a number between 0 and 5.")